In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [19]:
import pandas as pd
df = pd.read_csv('/content/gdrive/My Drive/Mestrado/second_level_pre.csv')

In [20]:
from ast import literal_eval

df['labels'] = df['labels'].apply(literal_eval)

In [21]:
df.shape

(202, 12)

In [22]:
df = df[df['labels'].apply(len) > 0]

In [23]:
import numpy as np
import pandas as pd
import random
from sklearn.datasets import make_classification, make_multilabel_classification
from sklearn.neighbors import NearestNeighbors
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

def create_dataset(df):

    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(df.labels)  
   
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['text'])
    df_countvect = pd.DataFrame(data = X.toarray())
    #y = pd.get_dummies(y, prefix='class')
    return  df_countvect, pd.DataFrame(y, columns=mlb.classes_), mlb



In [24]:
X, y, mlb = create_dataset(df)                     #Creating a Dataframe

In [25]:
label_configurations = np.array(list(set(list(map(tuple, y.values.tolist())))))

In [26]:
label_configurations

array([[1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 1],
       [0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 1],
       [0, 0, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1]])

In [27]:
X.shape, y.shape

((173, 2836), (173, 6))

In [28]:
label_counter = []
maximum_number = 0

for i in range(len(mlb.classes_)):
  current_number = sum(y.iloc[: , i])
  label_counter.append(current_number)
  maximum_number = max(maximum_number, current_number)

In [29]:
label_counter

[5, 12, 52, 3, 79, 47]

In [30]:
from scipy.special import softmax, expit
import random


In [31]:
np.std(label_counter)

28.278378077016132

In [58]:
iterations = 1

while iterations > 0 :

  IRPL = maximum_number / np.array(label_counter)

  #if iterations % 3 == 0:
    #print( np.std(IRPL))
    #print((IRPL > np.array(IRPL).mean()).sum())
  print(IRPL)
  # vectorization calculation
  print(label_configurations)
  intermediate_values = label_configurations * IRPL
  print(intermediate_values)
  average =np.sum(intermediate_values, axis=1) / np.sum(label_configurations, axis=1)
  print(average)
  assert len(average) == len(label_configurations)

  probabilities = average/sum(average)

  print(probabilities)

  acc = np.cumsum(probabilities)
  random_probability = random.uniform(0, 1)

  for i in range(len(acc)):
    upper_limit = acc[i]
    if random_probability <= upper_limit:
      index = i
      break

  labels_to_update = label_configurations[index]

  # then we need to update
  for i in range(len(labels_to_update)):
    if labels_to_update[i] == 1:
      label_counter[i] += 1
      maximum_number = max(label_counter[i], maximum_number)
  
  iterations = iterations - 1



[1.49982187 1.28255903 1.26217958 1.76779341 1.29538462 1.        ]
[[1 0 1 0 0 0]
 [0 1 0 0 1 1]
 [0 0 1 0 1 0]
 [0 1 0 0 1 0]
 [0 0 0 0 1 0]
 [1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 1 1]
 [1 0 0 0 0 1]
 [0 0 1 0 0 1]
 [0 0 1 1 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 0 1]
 [1 1 0 0 0 0]
 [0 1 0 0 0 1]
 [0 0 0 1 0 1]]
[[1.49982187 0.         1.26217958 0.         0.         0.        ]
 [0.         1.28255903 0.         0.         1.29538462 1.        ]
 [0.         0.         1.26217958 0.         1.29538462 0.        ]
 [0.         1.28255903 0.         0.         1.29538462 0.        ]
 [0.         0.         0.         0.         1.29538462 0.        ]
 [1.49982187 0.         0.         0.         0.         0.        ]
 [0.         0.         1.26217958 0.         0.         0.        ]
 [0.         1.28255903 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.29538462 1.        ]
 [1.49982187 0.         0.         0.         0.       

In [56]:
label_counter

[5614, 6564, 6671, 4763, 6499, 8420]

In [51]:
np.array([5, 12, 52, 3, 79, 47]) / sum([5, 12, 52, 3, 79, 47])

array([0.02525253, 0.06060606, 0.26262626, 0.01515152, 0.3989899 ,
       0.23737374])

In [55]:
np.array(label_counter) / sum(label_counter)

array([0.14570086, 0.17035634, 0.17313332, 0.12361475, 0.16866938,
       0.21852534])

In [53]:
print((IRPL > np.array(IRPL).mean()).sum())

2


In [ ]:
label_counter

[140, 154, 179, 122, 192, 238]

In [ ]:
!pip install textattack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 415 kB 4.7 MB/s 
     |████████████████████████████████| 4.2 MB 32.1 MB/s 
     |████████████████████████████████| 41.4 MB 1.4 MB/s 
     |████████████████████████████████| 3.6 MB 45.9 MB/s 
     |████████████████████████████████| 401 kB 40.9 MB/s 
     |████████████████████████████████| 769 kB 41.8 MB/s 
     |████████████████████████████████| 101 kB 9.8 MB/s 
     |████████████████████████████████| 290 kB 40.6 MB/s 
     |████████████████████████████████| 60 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 43.6 MB/s 
     |████████████████████████████████| 212 kB 34.2 MB/s 
     |████████████████████████████████| 1.1 MB 63.4 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 6.6 MB 45.7 MB/s 
     |████████████████████████████████| 596 kB 49.4 MB/s 
     |███████████████████████████████

In [ ]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 4.5 MB/s 
     |████████████████████████████████| 511.7 MB 5.8 kB/s 
     |████████████████████████████████| 1.6 MB 45.0 MB/s 
     |████████████████████████████████| 438 kB 51.9 MB/s 
     |████████████████████████████████| 5.8 MB 32.0 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: kera

In [ ]:
from textattack.augmentation import WordNetAugmenter
text = "start each day with positive thoughts and make your day"
wordnet_aug = WordNetAugmenter()
wordnet_aug.augment(text)

ImportError: ignored

In [ ]:
df['text'][2]

'There is nothing to be ashamed of. Everyone needs help at some point in their life. You have tried everything you can on your own. Now you need to trust yourself and work with a mental health professional. Medication can be extremely helpful if that is what you need.'

In [ ]:
wordnet_aug.augment(df['text'][2])

['There is nothing to be ashamed of. Everyone needs help at some manoeuvre in their life. You have tried everything you can on your own. Now you want to cartel yourself and work with a mental health professional. Medication can be extremely helpful if that is what you postulate.']

In [ ]:
from textattack.augmentation import EmbeddingAugmenter
embed_aug = EmbeddingAugmenter()
embed_aug.augment(text)

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:13<00:00, 36.9MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp4qds68rp.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


['initiating each day with positive thoughts and make your day']

In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embeddings = embed([ "The quick brown fox jumps over the lazy dog.", "I am a sentence for which I would like to get its embedding"])

print(embeddings)

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


Using /tmp/tfhub_modules to cache modules.
Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.0508086  -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [ ]:
import tensorflow as tf
import math
import numpy as np

In [ ]:
# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]
# Reduce logging output.
message_embeddings = embed(messages)

for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join((str(x) for x in        message_embedding[:3]))
        print("Embedding[{},...]\n".
                   format(message_embedding_snippet))
        print(np.std(message_embedding))

Message: Elephant
Embedding size: 512
Embedding[0.008344486355781555, 0.00048085825983434916, 0.06595248728990555,...]

0.04417864423599225
Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding[0.050808604806661606, -0.016524329781532288, 0.01573779620230198,...]

0.04396034306419817
Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding[-0.028332693502306938, -0.0558621808886528, -0.012941480614244938,...]

0.04409066512422514


In [ ]:
import seaborn as sns

In [ ]:
def similarity(text1, text2):
  sts_encode1 = tf.nn.l2_normalize(embed([text1]), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed([text2]), axis=1)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  """Returns the similarity scores"""
  return scores[0]

In [ ]:
from nltk.tokenize import sent_tokenize

def sentence_similarity(embed_aug, sentence1):
  sentences_tokenized = sent_tokenize(sentence1)
 
  similarities = []
  phrases = []
  for p1 in sentences_tokenized:
    
    best_one = 1
    for i in range(1):
      
      p2  = embed_aug.augment(p1)[0]
      if best_one > similarity(p1, p2):
        best_one = similarity(p1, p2)
        phrase = p2
    
    similarities.append(best_one)
    phrases.append(phrase)

  
  return sum(similarities) / len(similarities), ' '.join(phrases)



In [ ]:
from textattack.augmentation import EmbeddingAugmenter
augmenter = EmbeddingAugmenter()
s = 'What I cannot create, I do not understand.'
augmenter.augment(s)


['Whereof I cannot create, I do not understand.']

In [ ]:
text = df['text'][1]
sentence_similarity(text)

(<tf.Tensor: shape=(), dtype=float32, numpy=0.897921>,
 'Your English is wondrous. I think you potentially know yourself well enough that you keep returning to the issue being anxiety. You say your life is ruined and it\'s obvious you are insanely miserable, but then you says "I don\'t want to take strong medicine". You were willing to try pharmaceuticals for a stomach problem; you were willing to take recreational medicinal that have no FDA control on dosage or ingredients! Why would you not be willing to take drug for anxiety (or whatever a therapist diagnoses you with)?? It is an illness just like those other ones you were tested and addressed for. The only difference is this illness might be underlying some of the other bodily symptoms you are experiencing! The best treatment for sentimental illnesses is a combination of medication and therapy. Get a referral to a mental health professional and beginning on the path towards getting your life back!')

In [ ]:
print(text)

Your English is excellent. I think you probably know yourself well enough that you keep returning to the issue being anxiety. You say your life is ruined and it's obvious you are extremely miserable, but then you say "I don't want to take strong medicine". You were willing to try medicine for a stomach problem; you were willing to take recreational drugs that have no FDA control on dosage or ingredients! Why would you not be willing to take medication for anxiety (or whatever a therapist diagnoses you with)?? It is an illness just like those other ones you were tested and treated for. The only difference is this illness might be underlying some of the other physical symptoms you are experiencing! The best treatment for emotional illnesses is a combination of medication and therapy. Get a referral to a mental health professional and start on the path towards getting your life back!


In [ ]:
import random
def textattack_data_augment(data, target, texattack_augmenter):
  aug_data = []

  aug_label = []

  for text, label in zip(data, target):

    if random.randint(0,2) != 1:
      print("not extending")
      aug_data.append(text)
      aug_label.append(label)
      continue

    aug_list = texattack_augmenter.augment(text)
    aug_data.append(text)
    aug_label.append(label)
    aug_data.extend(aug_list)
    aug_label.extend([label]*len(aug_list))

  return aug_data, aug_label

In [ ]:
embed_aug = EmbeddingAugmenter(pct_words_to_swap=0.1, transformations_per_example=1)
aug_data, aug_lable = textattack_data_augment(df["text"], df["labels"], embed_aug)

# Paraphrasing

In [ ]:
! pip install git+https://github.com/PrithivirajDamodaran/Parrot.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PrithivirajDamodaran/Parrot.git to /tmp/pip-req-build-bpmtouik
  Running command git clone -q https://github.com/PrithivirajDamodaran/Parrot.git /tmp/pip-req-build-bpmtouik
     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 41.9 MB/s 
     |████████████████████████████████| 50 kB 5.9 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 29.9 MB/s 
     |████████████████████████████████| 596 kB 11.0 MB/s 
  Created wheel for parrot: filename=parrot-1.0-py3-none-any.whl size=8620 sha256=85d2537d6b7ef83514571b7c75e163c4a463d069ee06355a6d837266595baa09
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nvxr0k4/wheels/d8/27/d8/b757755a5b0b804c59714fdd8a4e2733c20802f638f17f7d96
  Created wheel for python-

In [ ]:
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# For reproducibility
def random_state(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

random_state(1234)

In [ ]:
#Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
phrases = [df['text'][2]]

In [ ]:
df['text'][2]

'There is nothing to be ashamed of. Everyone needs help at some point in their life. You have tried everything you can on your own. Now you need to trust yourself and work with a mental health professional. Medication can be extremely helpful if that is what you need.'

In [ ]:
for phrase in phrases:
  print("-"*100)
  print("Input_phrase: ", phrase)
  print("-"*100)
  para_phrases = parrot.augment(input_phrase=phrase)
  for para_phrase in para_phrases:
   print(para_phrase)

----------------------------------------------------------------------------------------------------
Input_phrase:  There is nothing to be ashamed of. Everyone needs help at some point in their life. You have tried everything you can on your own. Now you need to trust yourself and work with a mental health professional. Medication can be extremely helpful if that is what you need.
----------------------------------------------------------------------------------------------------
('There is nothing to be ashamed of. Everyone needs help at some point in their life. You have tried everything you can on your own. Now you need to trust yourself and work with a mental health professional. Medication can be extremely helpful if that is what you need.', 0)


In [ ]:
[0,1,2][0:3]

[0, 1, 2]

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
import numpy as np
nltk.download('punkt')

def augment_sample(text, augmenter):
  n = len(sent_tokenize(text))

  result = []
  smallest = 1000

  for sentence in sent_tokenize(text):
    sentence_augments = augmenter.augment(input_phrase=sentence)

    if sentence_augments is None:
      divide = sentence.split(',')

      for sub_sentence in divide:
        sub_sentence_augments = augmenter.augment(input_phrase=sub_sentence)
        smallest = min(smallest, len(sub_sentence_augments))
        result.append(sub_sentence_augments[0: smallest])
      
    else:
      smallest = min(smallest, len(sentence_augments))
      result.append(sentence_augments[0: smallest])

  
  for i in range(len(result)):
    result[i] = [a_tuple[0] for a_tuple in result[i][0:smallest]]

  
  result = np.array(result).T
  ans = []
  for i in range(len(result)):
    ans.append(' '.join(result[i]))
    #result[i] = ' '.join(result[i])
      
  return ans

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
%%time
result = augment_sample(df['text'][0], parrot)

At first
[('At first', 0)]
************
 it was not something that avoided me to do things
[("it wasn't something which prevented me from doing anything", 35), ("there's nothing that prevented me from doing things", 34), ("it wasn't something that scared me", 33), ('it was not something that prevented me from doing it', 28), ("it wasn't something that kept me from doing things", 28), ("it wasn't something that prevented me from doing things", 27), ("it wasn't something that taped me from doing things", 26), ("it wasn't something that stopped me from doing things", 25)]
************
 like keep doing the regular things in my life
[('keep doing the regular things in my life', 18), ('just keep doing the regular things of my life', 18), ('like keep doing the usual things in my life', 18)]
************
 but once I got back to Brazil in february of 2008
[('but once i returned to brazil in february 2008 i was really in love with brazil', 55), ('but when i came back to brazil in february 2008 i

In [ ]:
result

['Hello! i\'m brazilian i\'m 27 years old my life is going to ruin me I always had a great stomach, I don\'t throw up since I was like 4 years old I think, and since I can remember I have always been afraid of vomiting too. I started to feel sick 2 years ago, december 2007 while I was living in the States, I started to feel dizzy and then nauseous. At first it wasn\'t something which prevented me from doing anything keep doing the regular things in my life but once i returned to brazil in february 2008 i was really in love with brazil every day i started to feel nauseous then you\'ll be quite disoriented sometimes i didn\'t feel as though i was crying  2 or even 3 weeks but then he\'s back it\'s back to life i went to several doctors in the usa and to a hospital in brazil I went to a gastroenterologist, did a endoscopy, and the doctor said that it was an infection somewhere on the way out of the stomach, prescribed me some medicine, I took it for some months but it didn\'t help at all.